##雲端硬碟設定指向

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls 'drive'

'My Drive'  'Shared drives'


In [3]:
import os
path = "/content/drive/My Drive/Colab Notebooks/Bert/AML/"
os.chdir(path)

In [4]:
!ls

 albert      trained_model4			 '名字分類(變成姓名學了).ipynb'
 data	     trained_model5			 '名字預測(變成姓名學了).ipynb'
 data.zip    使用word2vec结合Textrank模型.ipynb   新聞分類訓練.ipynb
 NER.ipynb   分類250.ipynb			  新聞分類預測.ipynb


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
import torch
from torch.utils.data import TensorDataset
import pickle

In [7]:
import sys 
sys.path.append('.')
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '.']

In [8]:
def use_model(model_name, config_file_path, model_file_path, vocab_file_path, num_labels):
    # 選擇模型並加載設定
    if(model_name == 'bert'):
        from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
        model_config, model_class, model_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
        tokenizer = model_tokenizer(vocab_file=vocab_file_path)
        return model, tokenizer
    elif(model_name == 'albert'):
        from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForSequenceClassification
        model_config, model_class, model_tokenizer = (AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, config=config)
        tokenizer = model_tokenizer.from_pretrained(vocab_file_path)
        return model, tokenizer

## 定義函式-快速建立BERT INPUT( tokenizer.build_inputs_with_special_tokens() )

In [9]:
def to_bert_ids(tokenizer,q_input):
    # 將文字輸入轉換成對應的id編號
    #快速建立BERT INPUT
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q_input)))

In [10]:
class DataDic(object):
    def __init__(self, answers):
        self.answers = answers #全部答案(含重複)
        self.answers_norepeat = sorted(list(set(answers))) # 不重複
        self.answers_types = len(self.answers_norepeat) # 總共多少類
        self.ans_list = [] # 用於查找id或是text的list
        self._make_dic() # 製作字典
    
    def _make_dic(self):
        for index_a,a in enumerate(self.answers_norepeat):
            if a != None:
                self.ans_list.append((index_a,a))

    def to_id(self,text):
        for ans_id,ans_text in self.ans_list:
            if text == ans_text:
                return ans_id

    def to_text(self,id):
        for ans_id,ans_text in self.ans_list:
            if id == ans_id:
                return ans_text

    @property
    def types(self):
        return self.answers_types
    
    @property
    def data(self):
        return self.answers

    def __len__(self):
        return len(self.answers)

In [11]:
from torch.utils.data import DataLoader
import torch

In [12]:
! pip install snownlp

     |████████████████████████████████| 37.6MB 81kB/s 
  Created wheel for snownlp: filename=snownlp-0.12.3-cp36-none-any.whl size=37760958 sha256=a94f48127a04a64db8400baa11e264008f53cc66bad4d0929e84b3aa84c6fb92
  Stored in directory: /root/.cache/pip/wheels/f3/81/25/7c197493bd7daf177016f1a951c5c3a53b1c7e9339fd11ec8f
Successfully built snownlp


In [13]:
from snownlp import SnowNLP

In [14]:
!pip install -U ckiptagger[tfgpu,gdown]

     |████████████████████████████████| 411.0MB 41kB/s 
     |████████████████████████████████| 3.8MB 39.1MB/s 
     |████████████████████████████████| 512kB 50.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=14fb56bceec0d99ac03138294680ad3c3f7ef08559160a7ed7015d410ebea6b3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling te

In [15]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [16]:
def load_data():
    # 使用 GPU：
    #    1. 安裝 tensorflow-gpu (請見安裝說明)
    #    2. 設定 CUDA_VISIBLE_DEVICES 環境變數，例如：os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    #    3. 設定 disable_cuda=False，例如：ws = WS("./data", disable_cuda=False)
    # 使用 CPU：
    global ws, pos,ner
    ws = WS("./data", disable_cuda=False)
    pos = POS("./data", disable_cuda=False)
    ner = NER("./data", disable_cuda=False)

def get_person_str(context):
  
    load_data()

    sentence_list = [context]

    word_sentence_list = ws(
        sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
        # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
        # coerce_dictionary = dictionary2, # words in this dictionary are forced
    )

    pos_sentence_list = pos(word_sentence_list)
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    #print(entity_sentence_list)

    set_entity=set()    
    for i, sentence in enumerate(sentence_list):
        #print()
        #print(f"'{sentence}'")
        #print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
        for entity in sorted(entity_sentence_list[i]):
              if entity[2] == 'PERSON':
                #print(entity[3])
                set_entity.add(entity[3].replace(" ", ""))
    str_entity = ','.join(set_entity)
    #print(str_entity)
    return str_entity.split(",")

In [17]:
# load and init
pkl_file = open('trained_model5/data_features.pkl', 'rb')
data_features = pickle.load(pkl_file)
answer_dic = data_features['answer_dic']

In [21]:
model_setting2 = {
    "model_name":"albert", 
    "config_file_path":"trained_model5/config.json", 
    "model_file_path":"trained_model5/pytorch_model.bin", 
    "vocab_file_path":"albert/albert_tiny/vocab.txt",
    "num_labels":2 # 分幾類
}    
model, tokenizer = use_model(**model_setting2)
model.eval()

#
q_inputs = [u'高雄近日爆發「直播主之亂」 ，知名直播主連千毅與寵物店老闆兼直播主鄭又仁槓上，引發砸店、開槍風波，昨日雙雙落網，今早分別進行偵訊。如今傳出連千毅直播販售的「賓士行李箱」疑為假貨，高雄檢警20日前往倉庫搜索將證物帶回，連千毅連忙開直播澄清，強調自己是代銷商，更秀出簡體字授權函，強調絕對是正品，但卻遭台灣賓士打臉，直指「德國奔馳戴姆勒公司」不存在！▲連千毅如今傳出賣假貨疑雲。（圖／翻攝臉書）連千毅又被封為「直播天王」，短短幾分鐘就能賣出跑車、直升機等，成功在業界闖出一片天，多次登上媒體版面，但卻因行事作風太高調，引來黑道注目，不僅衝突頻傳，如今更傳出他所賣的「賓士行李箱」是假貨。高雄檢警20日上午會同保二總隊、經發局等單位，聯合稽查搜索連千毅的倉庫直播站，當場查扣一些賓士行李箱等商品，疑似有仿冒嫌疑，將其帶回調查，連千毅則大動作維護商譽，強調自己是代銷商，「全亞洲不是只有你賣賓士！」、「今天台灣授權廠商有講，未經他們認同的都非正貨，路上10台有8台是水貨車，你能說他是山寨仿冒嗎？」並揚言要對不實媒體提告。▲連千毅強調自己只是代銷商。（圖／翻攝臉書）連千毅秀出賓士授權函，強調產品是正品，但卻有眼尖網友發現疑點，除了中文部分皆用簡體字外，連授權書的英文都拼錯，將「AUTHORIZATION」打成「AUTHORIZAT『T』ON」，此外授權方「茲」也不知道是誰，台灣賓士對此回應：「我們戴姆勒集團旗下子公司，沒有一個叫做『德國奔馳戴姆勒公司』，並沒有這樣的公司存在。」據《ETtoday》報導，追查同款式行李箱，發現連千毅在直播上喊價拍賣上，知名購物平台也有同款商品，要價3萬9千元，但連千毅卻從6000元開始喊價，價差相當之大，如今傳出該款行李箱為仿冒品後，商品已從該網站悄悄下架， 避免衍生風波。']

for q_input in q_inputs:
    old_q_input=q_input
    if len(tokenizer.tokenize(q_input)) >511 :
      q_input=q_input[0:510]
    bert_ids = to_bert_ids(tokenizer,q_input)
    print('len(bert_ids)',len(bert_ids))
    assert len(bert_ids) <= 512
    input_ids = torch.LongTensor(bert_ids).unsqueeze(0)

    # predict
    outputs = model(input_ids)
    predicts = outputs[:2]
    predicts = predicts[0]
    max_val = torch.max(predicts)
    label = (predicts == max_val).nonzero().numpy()[0][1]
    ans_label = answer_dic.to_text(label)
    
    print(q_input)
    print(ans_label)
    if ans_label =='0':
        print('[]');
    else:
        snow_nlp = SnowNLP(old_q_input)
        snow_summer_list=snow_nlp.summary(10)
        snow_summer_str = ",".join(snow_summer_list)
        print(get_person_str(snow_summer_str)) #['馬勝', '安東尼羅賓', '廖泰宇']

len(bert_ids) 487
前新北市副市長許志堅2015年因收受建商賄賂 ，幫助建商快速通過都更案，今年3月判刑10年定讞並入監執行。板橋區介壽段與新店區廣明段2筆都更案在他涉賄案發後，市府皆暫停審議程序，至今停擺3年餘。依法院判決結果，許志堅違反貪污治罪條例不違背職務收受賄賂罪，今年7月初市府都市更新處發文給申請人，通知案件可繼續審議。板橋區介壽段都更案位於中山路1段、中山路1段50巷與民族路街廓，距捷運府中站300公尺，鄰近熱鬧的府中商圈，基地面積約1275平方公尺，其中市有地面積1266平方公尺，公有地佔整體比率99％，土地使用分區為商業區，預計興建地上19層大樓。2013年財政局辦理公開甄選作業，由樂揚建設取得最優申請人資格。請繼續往下閱讀...新店廣明段都更案位於新店區北宜路1段、能仁路口，鄰近新店國小，基地面積5951平方公尺，距捷運新店站600公尺，土地使用分區為住宅區，範圍內多為老舊磚造或鐵皮屋等建物，巷道狹小，寶興建設2008年起申請都更，預計興建地上29層大樓。都市更新處主任秘書李擇仁表示，當初2案進入司法調查階段，因此暫停審議程序，依據高等法院判決結果，許志堅違反貪污治罪條例不違背職務收賄罪，意為行賄事實與案件
1
['許志堅']
